In [5]:
import os

In [6]:
%pwd

'c:\\Users\\Arun Singh Negi\\Documents\\text_summarization\\research'

change directory

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\Arun Singh Negi\\Documents\\text_summarization'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from TextSummariser.constants import *
from TextSummariser.utils.common import read_yaml,create_directories

In [11]:
class ConfiguartionManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            parama_file_path=PARAMS_FILE_PATH,
    ):
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(parama_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
        


In [12]:
import urllib.request as request
import zipfile
from TextSummariser.logging import logger
from TextSummariser.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    
    def download_file(self):

        """
        Description: This function download zip filefrom given url

        Return: None


        """
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )

            logger.info(f"{filename} downlaod! with following info: \n{headers}")
        else:
            logger.info(f"File alreay existof size {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        Description: This function extract elements of a zi file.

        Return: None
        
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)    

In [16]:
try:
    config=ConfiguartionManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-02-05 16:24:06,658: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-02-05 16:24:06,667: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-02-05 16:24:06,670: INFO: common: created directory at : artifacts]
[2024-02-05 16:24:06,673: INFO: common: created directory at : artifacts/data_ingestion]
[2024-02-05 16:24:21,835: INFO: 3377218121: artifacts/data_ingestion/data.zip downlaod! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 763C:45C0:1CDF2B:2221A2:65C0BE4F
Accept-Ranges: bytes
Date: Mon, 05 Feb 2024 10:54:08 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10232-MAA
X-Cache: MIS